In [ ]:
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, figure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, UpSampling2D
from tensorflow.keras import backend as K
import random
import glob
import os
import cv2
from PIL import Image
import pixellib


from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from seaborn import color_palette

In [ ]:
!python -m pip install seaborn

In [ ]:
!pip install tensorflow-object-detection-api

In [ ]:
!pip install --ignore-installed --upgrade tensorflow==2.5.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
vidcap = cv2.VideoCapture('Indian Road accidents compilation 2021_480p.mp4')
vidcap

In [ ]:
try:
    # creating a folder named data
    if not os.path.exists('data'):
        os.makedirs('data') 
#if not created then raise error
except OSError:
    print ('Error: Creating directory of data')

In [ ]:
  #frame
currentframe = 0
while(True):
      
    # reading from frame
    ret,frame = vidcap.read()
  
    if ret:
        # if video is still left continue creating images
        name = './data/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
    else:
        break

In [ ]:
# # # Release all space and windows once done
vidcap.release()
cv2.destroyAllWindows()

In [ ]:
# ## Splitting image data to train, test, and valid data
!pip install split-folders tqdm
import splitfolders  
input_folder= 'data1/'
output= 'out_data/'

# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output, seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

splitfolders.fixed(input_folder, output, seed=1337, fixed=(100, 100), oversample=False, group_prefix=None) 

In [ ]:
img = plt.imread('data1\data\\frame1.jpg')   # reading image using its name
plt.imshow(img)

In [ ]:
## Splitting image data to train, test, and valid data
import splitfolders  
input_folder= 'df/'
output= 'modeldf/'

# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.7, .3)`.
splitfolders.ratio(input_folder, output, seed=1337, ratio=(.7, .2, .1), group_prefix=None) # default values

splitfolders.fixed(input_folder, output, seed=1337, fixed=(100, 100), oversample=False, group_prefix=None) 

In [ ]:
len('modeldf/train/Accidenthumanerror/')

In [ ]:
batch_size = 150
img_height = 256
img_width = 256

In [ ]:
from tensorflow.python import keras

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
    shear_range = 0.2,      
    zoom_range = 0.2,
    horizontal_flip = True) 

In [ ]:
train_generator = train_datagen.flow_from_directory('modeldf/',target_size = (256, 256),
                                                    color_mode = "rgb")
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
validation_generator = test_datagen.flow_from_directory('modeldf/',target_size = (256, 256),batch_size = 15,color_mode = 'rgb')

In [ ]:
training_d = tf.keras.preprocessing.image_dataset_from_directory(
    'modeldf/train/',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size)

In [ ]:
validation =  tf.keras.preprocessing.image_dataset_from_directory(
    'modeldf/val/',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size)

In [ ]:
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'modeldf/test',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size)

class_names = training_d.class_names

## Configuring dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
training_ds = training_d.cache().prefetch(buffer_size=AUTOTUNE)
testing_ds = testing_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras import layers
Cnn = tf.keras.models.Sequential([
  layers.BatchNormalization(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(len(class_names),activation= 'sigmoid')
])

Cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
retVal = Cnn.fit(training_ds, validation_data= validation, epochs = 10)

In [ ]:
plt.plot(retVal.history['loss'], label = 'training loss')
plt.plot(retVal.history['accuracy'], label = 'training accuracy')
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(retVal.history['val_loss'], label = 'validation loss')
plt.plot(retVal.history['val_accuracy'], label = 'validation accuracy')
plt.grid(True)
plt.legend()

In [ ]:
## lets vizualize results on testing data
AccuracyVector = []
plt.figure(figsize=(30, 30))
for images, labels in testing_ds.take(1):
    predictions = MyCnn.predict(images)
    predlabel = []
    prdlbl = []
    
    for mem in predictions:
        predlabel.append(class_names[np.argmax(mem)])
        prdlbl.append(np.argmax(mem))
    
    AccuracyVector = np.array(prdlbl) == labels
    for i in range(40):
        ax = plt.subplot(10, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title('Pred: '+ predlabel[i]+' actl:'+class_names[labels[i]] )
        plt.axis('off')
        plt.grid(True)

In [ ]:
Cnn.save('Model.h5')

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(MyCnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)